In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from utils import *
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 100)
matplotlib.rcParams.update({"font.size": 16,'lines.linewidth': 2.5})
# matplotlib.rcdefaults()

figures saved to ../tmp/figures


In [4]:
DATA_DIR = '../data/'
dfj = get_df(DATA_DIR + 'pai_job_table.csv')
dft = get_df(DATA_DIR + 'pai_task_table.csv')
dfi = get_df(DATA_DIR + 'pai_instance_table.csv')
dfs = get_df(DATA_DIR + 'pai_sensor_table.csv')
dfg = get_df(DATA_DIR + 'pai_group_tag_table.csv')
dfp = get_df(DATA_DIR + 'pai_machine_spec.csv')
dfm = get_df(DATA_DIR + 'pai_machine_metric.csv')

In [8]:
# 假设 'df' 是通过 get_df 函数获取到的 DataFrame 对象
unique_values_task_name = dft['task_name'].unique()
print("task_name:\n",unique_values_task_name)
unique_values1_gpu_type_spec = dfg['gpu_type_spec'].unique()
print("gpu_type_spec:\n",unique_values1_gpu_type_spec)
unique_values_workload = dfg['workload'].unique()
print("workload:\n",unique_values_workload)

task_name:
 ['tensorflow' 'worker' 'PyTorchWorker' 'xComputeWorker' 'ps' 'ReduceTask'
 'TensorboardTask' 'evaluator' 'TVMTuneMain' 'DecoderWorker'
 'OpenmpiTracker' 'OpenmpiWorker' 'JupyterTask' 'OssToVolumeWorker'
 'chief' 'MWorker' 'TransformGraph' 'BladeMain' 'aligraph' 'M1' 'R3_1_2'
 'M2' 'R2_1' 'TfClientWorker']
gpu_type_spec:
 [nan 'V100' 'P100' 'V100M32' 'T4' 'MISC']
workload:
 [nan 'bert' 'xlnet' 'nmt' 'graphlearn' 'resnet' 'ctr' 'inception' 'vgg'
 'rl']
